In [2]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 47 kB/s 
     |████████████████████████████████| 199 kB 47.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=38e1882b382c50f705e2b8132b57bc789aa8acf743c559e2eeb16fa173a31d3d
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


Importing modules not related to Pyspark

In [3]:
import os
import sys
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
import math
from IPython.core.interactiveshell import InteractiveShell
from datetime import *
import statistics as stats
InteractiveShell.ast_node_interactivity = "all" 
%matplotlib inline

Importing Pyspark modules


In [6]:
import pyspark
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions
from pyspark.sql.functions import lit, desc, col, size, array_contains\
, isnan, udf, hour, array_min, array_max, countDistinct
from pyspark.sql.types import *
MAX_MEMORY = '15G'
# Initialize a spark session.
conf = pyspark.SparkConf().setMaster("local[*]") \
        .set('spark.executor.heartbeatInterval', 10000) \
        .set('spark.network.timeout', 10000) \
        .set("spark.core.connection.ack.wait.timeout", "3600") \
        .set("spark.executor.memory", MAX_MEMORY) \
        .set("spark.driver.memory", MAX_MEMORY)
def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Pyspark guide") \
        .config(conf=conf) \
        .getOrCreate()
    return spark

spark = init_spark()
df = spark.read.csv('/home/Violence Against Women  Girls Data.csv',inferSchema=True, header=True)
print('Data frame type: ' + str(type(df)))

Data frame type: <class 'pyspark.sql.dataframe.DataFrame'>


In [7]:
print('Data frame describe (string and numeric columns only):')
df.describe().toPandas()

print(f'There are total {df.count()} row, Let print first 2 data rows:')
df.limit(2).toPandas()

Data frame describe (string and numeric columns only):


,summary,RecordID,Country,Gender,Demographics Question,Demographics Response,Question,Survey Year,Value
0,count,12020,12020,12020,12020,12020,12020,12020,10697
1,mean,208.8785357737105,None,None,None,None,None,None,19.696952416565377
2,stddev,121.11855015561693,None,None,None,None,None,None,17.132523298852146
3,min,1,Afghanistan,F,Age,15-24,... for at least one specific reason,01/01/2000,0.0
4,max,417,Ukraine,M,Residence,"Widowed, divorced, separated",... if she refuses to have sex with him,01/01/2018,86.9


There are total 12020 row, Let print first 2 data rows:


,RecordID,Country,Gender,Demographics Question,Demographics Response,Question,Survey Year,Value
0,1,Afghanistan,F,Marital status,Never married,... if she burns the food,01/01/2015,NaN
1,1,Afghanistan,F,Education,Higher,... if she burns the food,01/01/2015,10.1


In [8]:
print('Columns overview')
pd.DataFrame(df.dtypes, columns = ['Column Name','Data type'])

Columns overview


,Column Name,Data type
0,RecordID,int
1,Country,string
2,Gender,string
3,Demographics Question,string
4,Demographics Response,string
5,Question,string
6,Survey Year,string
7,Value,double


In [9]:
string_columns = ['Country', 'Gender', 'Demographics Question', 'Demographics Response', 'Question', 'Survey Year']
numeric_columns = ['RecordID','Value']
missing_values = {} 
for index, column in enumerate(df.columns):
    if column in string_columns:
        missing_count = df.filter(col(column).eqNullSafe(None) | col(column).isNull()).count()
        missing_values.update({column:missing_count})
    if column in numeric_columns:  
        missing_count = df.where(col(column).isin([0,None,np.nan])).count()
        missing_values.update({column:missing_count})
missing_df = pd.DataFrame.from_dict([missing_values])
missing_df

,RecordID,Country,Gender,Demographics Question,Demographics Response,Question,Survey Year,Value
0,0,0,0,0,0,0,0,25


In [17]:
df = df.withColumn('ViolenceRecordCount',col('RecordID'))

def Violence_record_summarize(df):
    victims_count = format(df.select('RecordID').distinct().count(), ',d')
    country_count = str(df.select('Country').distinct().count())
    sum_temp = df.agg(functions.sum('ViolenceRecordCount')).toPandas()
    total_records_count = format(sum_temp['sum(ViolenceRecordCount)'][0],',d')
    columns=['Victims count', 'Country types count', 'Total records count']
    data = [[victims_count], [country_count], [total_records_count]]
    sum_dict = {column: data[i] for i, column in enumerate(columns)}
    sum_df = pd.DataFrame.from_dict(sum_dict)[columns]
    gender_user_count = df.select('Gender','RecordId').distinct().groupBy('Gender').count().toPandas()
    gender_activities_count = df.groupBy('Gender').count().toPandas()
    gender_user_activity_count = gender_user_count.join(
        gender_activities_count.set_index('Gender'), on='Gender'
        , how='inner', lsuffix='_gu'
    )
    gender_user_activity_count.columns = ['Gender', 'No. of users', 'Violence Record count']
    
    return sum_df, gender_user_activity_count

sum_dfs= Violence_record_summarize(df)
print('\nOverall data set summary on No of users, cases and number of records (pre-filtering):')
sum_dfs[0]


Overall data set summary on No of users, cases and number of records (pre-filtering):


,Victims count,Country types count,Total records count
0,402,67,"2,510,720"


In [ ]:
victims_per_country_df = df.select(df.RecordID, df.Country) \
    .distinct() \
    .groupBy(df.Country) \
    .count() \
    .orderBy("count", ascending=False)

# Top 5 workout types
highest_sport_users_df = ranked_sport_users_df.limit(5).toPandas()
# Rename column name : 'count' --> Users count
highest_sport_users_df.rename(columns = {'count':'Users count'}, inplace = True)
# Caculate the total users, we will this result to compute percentage later
total_sports_users = ranked_sport_users_df.groupBy().sum().collect()[0][0]